This notebook uses code snippets from the crocodile python examples (https://github.com/SKA-ScienceDataProcessor/crocodile) to perform w-projection convolutional gridding of a set of continuous visibilities, followed by a Fourier Transform the grid to form a Dirty Image.

To run the script you will need:
 - ipython
 - numpy
 - pylab
 - simulated_data.txt (or any other set of visibility data stored as a text file with columns (u, v, w, real(V), imag(V)).
 
The first three of these can be easily obtained using pip. 

To get pip: 

> apt-get install python-pip

To install each library:

>pip install numpy

etc.

The fourth is a text file created by Simulate_uvw.ipynb. This script assumes it is in the working directory. 


Set up python stuff...

In [ ]:
%matplotlib inline

In [ ]:
import numpy
import matplotlib.pyplot as pl
pl.rcParams['figure.figsize'] = 16, 12

First we're going to define some user-supplied parameters:

1. The w-increment
2. The kernel over-sampling factor
3. The kernel support size
4. The kernel far-field size 

In [ ]:
wstep=300
over_sampling=8
ff_size=256
kernel_support=7

Then we're going to get some visibility data. You can create these data using the Simulate_uvw.ipynb notebook.

In [ ]:
u,v,w,vis_re,vis_im = numpy.loadtxt('./simulated_data.txt',unpack=True)
vis = vis_re + 1j*vis_im
uvw = numpy.column_stack((u,v,w))

# conjugate symmetry
tmp_uvw = uvw*numpy.array([-1.,-1.,1.])
tmp_vis = vis_re - 1j*vis_im

vis = numpy.hstack((vis,tmp_vis))
uvw = numpy.vstack((uvw,tmp_uvw))

pl.subplot(111)
pl.scatter(uvw[:,0],uvw[:,1])
pl.show()
print "Range of w-values:",numpy.amin(uvw[:,2])," - ",numpy.amax(uvw[:,2])," lambda"

Image parameters are defined in terms of the half-width of the FOV, T2, and the half-width of the uv-plane, L2, which is used as an approximation to the angular resolution. 

The number of pixels on the side of an image is 

$$N_{\rm pix} = \frac{\Theta_{\rm FOV}} {\theta_{\rm res}}.$$

Therefore the number of pixels along a side is given by 

$$N_{\rm pix} = 2 T_{1/2}\times 2 L_{1/2}.$$

In [ ]:
T2 = 0.002 # half-width of FOV [radians]
L2 = 30000 # half-width of uv-plane [lambda]
N = int(T2*L2*4) # number of pixels 
print "Making grids of side: ",N," pixels."

Create an empty grid to grid visibilities onto, as well as a grid for weights to make the synthesized beam:

In [ ]:
grid_uv=numpy.zeros([N, N], dtype=complex)
grid_wt=numpy.zeros([N, N], dtype=complex)

Sort the uvw and visibility data by w:

In [ ]:
temp=uvw
zs=numpy.argsort(uvw[:,2])
uvw = uvw[zs]
vis = vis[zs]

pl.subplot(121)
pl.plot(temp[:,2])
pl.title("Before")
pl.subplot(122)
pl.plot(uvw[:,2])
pl.title("After")
pl.show()

Then create an array of w-planes spanning the range of w -values in the data:

In [ ]:
ii=range(0, len(vis), wstep)
print ii

Create a list of boundaries in w-value for each plane:

In [ ]:
ir=zip(ii[:-1], ii[1:]) + [(ii[-1], len(vis))]
print ir

Each set of limits constitutes a w-plane. With the limits:

In [ ]:
ilow,ihigh=ir[0]

For each plane we need to calculate the mean w-value in the uvw-data:

In [ ]:
w=uvw[ilow:ihigh,2].mean()
print w

Set up some drawing tools that will be useful in a second...

In [ ]:
def show_image(ff, name, w_dep=False):
    # Determine mid position and size of image
    size = ff.shape[0]/2
    lm_size = 2*size*T2/N
    pl.subplot(121)
    pl.imshow(ff.real, extent=(-lm_size,lm_size,-lm_size,lm_size))
    pl.title("$%s(l,m%s)$: Real" % (name, ",w" if w_dep else ""))
    pl.xlabel(r"L [$1$]"); pl.ylabel(r"M [$1$]")
    pl.subplot(122)
    pl.imshow(ff.imag, extent=(-lm_size,lm_size,-lm_size,lm_size))
    pl.title("$%s(l,m%s)$: Imag" % (name, ",w" if w_dep else ""))
    pl.xlabel(r"L [$1$]"); pl.ylabel(r"M [$1$]")
    pl.show()
def show_grid(af, name, norm=None, size=None, over=None):
    # Determine mid position and size of grid portion. Factor in oversampling.
    mid = af.shape[0]/2
    if size is None:
        size = af.shape[0]/2
    elif over is not None:
        size *= over
    uv_size = 2*size*L2/N    
    if over is not None:
        uv_size /= over
    # Determine normalisation for image.
    from  matplotlib.colors import Normalize
    if norm is not None:
        norm = Normalize(vmin=-norm, vmax=norm, clip=True)
    else:
        norm = None
    # Draw.
    for plot, comp, data in [(121, "Real", af.real), (122, "Imag", af.imag)]:
        pl.subplot(plot)
        pl.imshow(data[mid-size:mid+size+1,mid-size:mid+size+1],
                  extent=(-uv_size, uv_size, -uv_size, uv_size),
                  norm=norm)
        pl.title("$%s(u,v,w)$: %s" % (name, comp))
        pl.xlabel(r"U [$\lambda$]")
        pl.ylabel(r"V [$\lambda$]")
        # Only show color bar if we don't use the standard normalisation.
        if norm is None: pl.colorbar(shrink=.4,pad=0.025)
    pl.show()

We then need to caluclate the w-kernel for this w-value. First we create a grid for the kernel with values

$$r^2 = \ell^2 + m^2$$

In [ ]:
upfront_oversampling = over_sampling
ff = T2*numpy.mgrid[-upfront_oversampling:upfront_oversampling-2./ff_size:(ff_size*upfront_oversampling*1j),
                    -upfront_oversampling:upfront_oversampling-2./ff_size:(ff_size*upfront_oversampling*1j)]
r2 = (ff[0])**2 + (ff[1])**2
show_image(r2, 'R', w_dep=True)

Then calculate the value of the kernel at each point, 

$$G(\ell,m,w) = {\rm e}^{-2\pi i  \left[w( \sqrt{1-\ell^2-m^2} - 1 )\right] } $$

(Eq. 11; Cornwell+ 2008 http://arxiv.org/pdf/0807.4161v1.pdf )

In [ ]:
ph=w*(numpy.sqrt(1.-r2)-1.)
cp=numpy.exp(-2j*numpy.pi*ph)
show_image(cp, 'G', w_dep=True)

We need to pad this and FFT it in order to over-sample it in uvw-space to obtain the kernel:

$$ \tilde{G}(u,v,w) \approx \frac{i}{w}{\rm e}^{ \pi i \left[ \frac{u^2 +v^2}{w} \right] } $$

(Eq. 14; Cornwell+ 2008 http://arxiv.org/pdf/0807.4161v1.pdf , however using FFT we obtain a more accurate version here.)

In [ ]:
padff=numpy.pad(cp, 
    pad_width=int(ff_size*(over_sampling/upfront_oversampling-1.)/2.),
    mode='constant',
    constant_values=(0.0,))
show_image(padff, "G", w_dep=True)

af=numpy.fft.fftshift(numpy.fft.ifft2(numpy.fft.ifftshift(padff)))
show_grid(af, r"\tilde G", norm=.5, over=over_sampling, size=kernel_support)

At this point we can quickly check whether the kernel support is large enough to contain most of the kernel. If this fails, try increasing `kernel_support`.

In [ ]:
af_sum = af.sum()
print "Full sum:", af_sum
assert(abs(af_sum - 1.0) < 1e-13)

mid=over_sampling*ff_size/2; size = over_sampling*kernel_support
af_part_sum = af[mid-size:mid+size+1,mid-size:mid+size+1].sum()
print "Kernel sum:", af_part_sum, abs(af_part_sum - 1.0)
assert(abs(af_part_sum - 1.0) < 5e-4) # quality check - increase kernel_support if this fails

## Shift

We intend to shift the FoV by a certain $(\Delta l, \Delta m)$ for faceting. This requires phase-rotating the visibilities:

$$V^{shift}(u,v,w) = e^{2\pi i (u\Delta l + v\Delta m)}V(u,v,w)$$

In [ ]:
delta_l = 0.0000 # 2 * T2
delta_m = 0.0000

shift = numpy.exp(2j*numpy.pi* (uvw[:,0]*delta_l+uvw[:,1]*delta_m))
vis_shift = vis * shift

As well as updating the grid convolution function. This would be a simple shift in the image plane, but for quality it is probably better to do this afterwards. As we are going to FFT the convolution $V * \tilde G$, it is not particularly surprising that this yields the same transformation we did for the visibilities:

$$\tilde G^{shift}(u,v,w) = e^{2\pi i (u\Delta m + v\Delta l)} \tilde G(u,v,w)$$

In [ ]:
ff = L2/N * numpy.mgrid[-ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j),
                        -ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j)]
rot = numpy.exp(2j*numpy.pi*(ff[1] * delta_l + ff[0] * delta_m))
af_shift = rot * af

mid = ff_size*over_sampling/2
size = over_sampling*kernel_support

show_grid(rot, "rot", size=kernel_support, over=over_sampling)
show_grid(af_shift, r"\tilde G^{shift}", size=kernel_support, over=over_sampling, norm=.5)
show_grid(af_shift - af, r"\left(\tilde G^{shift} - \tilde G\right)", size=kernel_support, over=over_sampling)

show_image(numpy.fft.ifftshift(numpy.fft.fft2(numpy.fft.fftshift(af_shift))), r"G^{shift}", w_dep=True)

## Transform

We might also want to distort our image in a linear fashion in order to compensate for projection errors far from the phase centre:


$$I^{trans}(l,m) = I^{shift}(T_{2\times2}(l,m))$$
$$T_{2\times2} = \begin{pmatrix} t_{11} & t_{12} \\ t_{21} & t_{22} \end{pmatrix} $$

This comes down to a transformation of $uv$ coordinates:

$$V^{trans}(u,v,w) = V^{shift}\bigl(\left(T^T_{2\times2}(u,v)\right)^{-1},w\bigr)$$


In [ ]:
# Transformation matrix
import math
a = 0 * numpy.pi / 2; s = 1.0
t_11 = s * math.cos(a); t_12 = s * -math.sin(a)
t_21 = s * math.sin(a); t_22 = s * math.cos(a)

# Determinant, inverse transposed
det = float(t_11 * t_22 - t_12 * t_21)
tti_11 =  t_22 / det; tti_12 = -t_12 / det
tti_21 = -t_21 / det; tti_22 =  t_11 / det

# New UV coordinates
u_trans = tti_11 * uvw[:,0] + tti_12 * uvw[:,1]
v_trans = tti_21 * uvw[:,0] + tti_22 * uvw[:,1]
uvw_trans = numpy.array([u_trans, v_trans, uvw[:,2]]).T

# Also shift intensity
vis_trans = vis_shift / abs(det)

Updating the GCF is theoretically exactly as straightforward. All we need to do is:

$$\tilde G^{trans} = G^{shift}\bigl(\left(T^T_{2\times2}(u,v)\right)^{-1},w\bigr)$$

This is actually a non-trivial image transformation this time around. Here is a rough version:

In [ ]:
ff = numpy.mgrid[-float(ff_size):ff_size-2./over_sampling:(ff_size*over_sampling*1j),
                 -float(ff_size):ff_size-2./over_sampling:(ff_size*over_sampling*1j)]

print tti_11, tti_12
print tti_21, tti_22

xs = (over_sampling/2.*(ff_size + tti_11 * ff[1] + tti_12 * ff[0]))
ys = (over_sampling/2.*(ff_size + tti_21 * ff[1] + tti_22 * ff[0]))
xs = numpy.minimum(numpy.maximum(xs, 0), ff_size * over_sampling - 1)
ys = numpy.minimum(numpy.maximum(ys, 0), ff_size * over_sampling - 1)
xrem = numpy.remainder(xs,1)
yrem = numpy.remainder(ys,1)

ixs = xs.astype(int); iys = ys.astype(int)
def inc(i): return (i+1)%(ff_size*over_sampling)
af_trans =  (
  (1-yrem) * ((1-xrem) * af_shift[iys,        ixs] + xrem * af_shift[iys,        inc(ixs)]) +
     yrem  * ((1-xrem) * af_shift[inc(iys+1), ixs] + xrem * af_shift[inc(iys+1), inc(ixs)]))

show_grid(af_trans, r"\tilde G^{trans}", size=kernel_support, over=over_sampling, norm=.5)

We can easily check that at this point we have moved and transformed the image plane. We just have to reverse the FFT:

In [ ]:
show_image(numpy.fft.ifftshift(numpy.fft.fft2(numpy.fft.fftshift(af_trans))), "G^{trans}", w_dep=True)

However, it is more accurate if we go a step back, generate the original $G$ for the transformed coordinates, then re-do the shift in the new coordinate system. So basically we do:

$$G^{trans-shift}(l,m,w) = G(T_{2\times2}(l,m),w)$$

followed by:

$$\tilde G^{shift}(u,v,w) = e^{2\pi i (u,v) \cdot T_{2\times2}(\Delta l, \Delta m)} \tilde G^{trans-shift}(u,v,w)$$

In [ ]:
ff = T2*numpy.mgrid[-upfront_oversampling:upfront_oversampling-2./ff_size:(ff_size*upfront_oversampling*1j),
                    -upfront_oversampling:upfront_oversampling-2./ff_size:(ff_size*upfront_oversampling*1j)]
t_ls = t_11 * ff[1] + t_12 * ff[0]
t_ms = t_21 * ff[1] + t_22 * ff[0]
r2 = t_ls**2 + t_ms**2
ph=w*(1.-numpy.sqrt(1.-r2))
cp=numpy.exp(2j*numpy.pi*ph)
padff=numpy.pad(cp, 
    pad_width=int(ff_size*(over_sampling/upfront_oversampling-1.)/2.),
    mode='constant',
    constant_values=(0.0,))
show_image(padff, "G^{trans2}", w_dep=True)

In [ ]:
af_trans_shift=numpy.fft.fftshift(numpy.fft.ifft2(numpy.fft.ifftshift(padff)))
show_grid(af_trans_shift, r"\tilde G^{trans-shift}", size=kernel_support, over=over_sampling, norm=.5)

In [ ]:
ff = (1/T2/4) * numpy.mgrid[-ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j),
                            -ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j)]
t_delta_l = t_11 * delta_l + t_12 * delta_m
t_delta_m = t_21 * delta_l + t_22 * delta_m
rot = numpy.exp(2j*numpy.pi*(ff[1] * t_delta_l + ff[0] * t_delta_m)/det)
af_trans2 = rot * af_trans_shift
show_grid(rot, "rot", size=kernel_support, over=over_sampling)
show_grid(af_trans2, r"\tilde G^{trans2}", size=kernel_support, over=over_sampling, norm=.5)
show_grid(af_trans - af_trans2, r"\left(\tilde G^{trans} - \tilde G^{trans2}\right)", size=kernel_support, over=over_sampling)

In [ ]:
ff_back = numpy.fft.ifftshift(numpy.fft.fft2(numpy.fft.fftshift(af_trans)))
ff_back2 = numpy.fft.ifftshift(numpy.fft.fft2(numpy.fft.fftshift(af_trans2)))
pl.subplot(121)
pl.imshow(ff_back.real)
pl.title(r"$G^{trans}(l,m,w)$: Real")
pl.subplot(122)
pl.imshow(ff_back.imag)
pl.title(r"$G^{trans}(l,m,w)$: Imag")
pl.show()
pl.subplot(121)
pl.imshow(ff_back2.real)
pl.title(r"$G^{trans2}(l,m,w)$: Real")
pl.subplot(122)
pl.imshow(ff_back2.imag)
pl.title(r"$G^{trans2}(l,m,w)$: Imag")
pl.show()

Then just extract the central region with size equal to the kernel support. I've added a couple of options for this: (1) from crocodile; (2) adapted from ASKAPsoft.

In [ ]:
def exmid2(a, s):
    """Extract a section from middle of a map, suitable for zero frequencies at N/2"""
    cx=a.shape[0]/2
    cy=a.shape[1]/2
    return a[cy-s-1:cy+s,cx-s-1:cx+s]

# works in the opposite sense to ASKAPsoft, which calculates an under-sampled kernel and 
# interpolates.
def wextract(a, i, j, Qpx, s):
    """Extract the (ith,jth) w-kernel from the oversampled parent and normalise
    The kernel is reversed in order to make the suitable for
    correcting the fractional coordinates
    """
    x=a[j::Qpx, i::Qpx] # decimate the kernel
    x=x[::-1,::-1] # reverse the kernel

    # extract middle, normalise (don't update x, that would write a!)
    norm=float(Qpx*Qpx)
    #norm=1.0/x.sum()
    print x.sum()
    return norm*exmid2(x,s)

def askapsoft_decimate_n_extract(af, over_sampling, kernel_support):
    
    """
    Extracted and translated from
    AWProjectVisGridder.cc
    """
    
    # why is this normalization required..?
    rescale = over_sampling*over_sampling
            
    cSize = 2 * kernel_support + 1
    itsConvFunc=numpy.zeros((over_sampling, over_sampling, cSize, cSize), dtype=complex)
            
    for fracu in range(0,over_sampling):
        for fracv in range(0,over_sampling):
            
            # Now cut out the inner part of the convolution function and
            # insert it into the convolution function
            for iy in range(-kernel_support,kernel_support+1):
                for ix in range(-kernel_support,kernel_support+1):
                    
                    nx = af.shape[0]
                    ny = af.shape[1]
                    
                    # assumes support is the same for all w-planes:
                    xval = (ix) * over_sampling + fracu + nx / 2
                    yval = (iy) * over_sampling + fracv + ny / 2
                    
                    itsConvFunc[fracv, fracu, iy+cSize/2, ix+cSize/2] \
                            = rescale * af[xval, yval]
                        
    return itsConvFunc

# --------------------------
# --------------------------
# Option (1):
wg=[[wextract(af, i, j, over_sampling, kernel_support) for i in range(over_sampling)] for j in range(over_sampling)]
# Convert list to numpy array:
wg = numpy.array(wg)
# --------------------------
# Option (2):
#wg = askapsoft_decimate_n_extract(af, over_sampling, kernel_support)
# --------------------------


for fracu in range(0,over_sampling):
    show_grid(wg[0,fracu,:,:], r"\tilde G", size=kernel_support, norm=.5)


Do the same for $\tilde G^{shift}$. This is the kernel we are actually interested in, but constructing both gives us a chance to compare.

In [ ]:
wg_shift =[[wextract(af_shift, i, j, over_sampling, kernel_support) for i in range(over_sampling)] for j in range(over_sampling)]
wg_shift = numpy.array(wg_shift)
wgd = wg_shift-wg

show_grid(wgd[2,2,:,:], r"\tilde G^{shift}", size=kernel_support, norm=.5)

And again, this time for $\tilde G^{trans}$.

In [ ]:
wg_trans =[[wextract(af_trans, i, j, over_sampling, kernel_support) for i in range(over_sampling)] for j in range(over_sampling)]
wg_trans = numpy.array(wg_trans)
wg_trans2 =[[wextract(af_trans2, i, j, over_sampling, kernel_support) for i in range(over_sampling)] for j in range(over_sampling)]
wg_trans2 = numpy.array(wg_trans2)

show_grid(wg_shift[2,2,:,:], r"\tilde G^{shift}",size=kernel_support, norm=.5)
show_grid(wg_trans[2,2,:,:], r"\tilde G^{trans}",size=kernel_support, norm=.5)
show_grid(wg_trans2[2,2,:,:], r"\tilde G^{trans2}",size=kernel_support, norm=.5)

Take the complex conjugate,

In [ ]:
wg = numpy.conj(wg_trans2)
show_grid(wg[2,2], r"$G^{shift\ast}(u,v,w)", size=kernel_support, norm=.5)

Now we have our convolution kernel, we need to find out where we should grid our visibility data.

In [ ]:
def fraccoord(N, p, Qpx):
    """Compute whole and fractional parts of coordinates, rounded to Qpx-th fraction of pixel size
    :param N: Number of pixels in total 
    :param p: coordinates in range -1,1
    :param Qpx: Fractional values to round to
    """
    H=N/2
    x=(1+p)*H
    flx=numpy.floor(x + 0.5 /Qpx)
    fracx=numpy.around(((x-flx)*Qpx))    
    return (flx).astype(int), fracx.astype(int)

uvw_sub = uvw_trans[ilow:ihigh,:]/L2 # extract subarray for w-slice
(x, xf), (y, yf) = [fraccoord(grid_uv.shape[i], uvw_sub[:,i], over_sampling) for i in [0,1]]

Then we can convolve the data onto the grid:

In [ ]:
def convgridone(a, pi, fi, gcf, v):
    """Convolve and grid one visibility sample"""
    sy, sx= gcf[0][0].shape[0]/2, gcf[0][0].shape[1]/2
    v *= numpy.exp(2j*numpy.pi*(pi[0]-N/2))
    
    # NB the order of fi below 
    a[ pi[1]-sy: pi[1]+sy+1, pi[0]-sx: pi[0]+sx+1 ] += gcf[fi[1],fi[0]]*v
    return a

def gridone(a,p,v):
    """grid one visibility without convolution"""
    
    a[p[1],p[0]] += v
    
    return a

grid_uv=numpy.zeros([N, N], dtype=complex)
grid_wt=numpy.zeros([N, N], dtype=complex)
vis_sub = vis_trans[ilow:ihigh]
for i in range(len(vis_sub)):
#    gridone(grid_uv, (x[i],y[i]), vis_sub[i])
    convgridone(grid_uv,(x[i], y[i]), (xf[i], yf[i]), wg, vis_sub[i])
    convgridone(grid_wt,(x[i], y[i]), (xf[i], yf[i]), wg, 1.0+0j)

show_grid(grid_uv, "V")
show_grid(grid_wt, "V^{psf}")

Let's FT this grid to check it gives a sensible map...

In [ ]:
dty_image=numpy.fft.fftshift(numpy.fft.ifft2(numpy.fft.ifftshift(grid_uv)))
psf_image=numpy.fft.fftshift(numpy.fft.ifft2(numpy.fft.ifftshift(grid_wt)))

show_image(dty_image, name="I")
show_image(psf_image, name="I^{psf}")

Done!

In [ ]:
dty_image_last = dty_image
psf_image_last = psf_image

In [ ]:
signal = psf_image[(psf_image.shape[0]/2-20):(psf_image.shape[0]/2+20),
                   (psf_image.shape[1]/2-20):(psf_image.shape[1]/2+20)]

print numpy.sum(numpy.abs(signal)) / (numpy.sum(numpy.abs(psf_image)) - numpy.sum(numpy.abs(signal)))

pl.subplot(121)
pl.imshow(signal.real)
pl.title("PSF Signal")
pl.show()

In [ ]:
grid_uv_o=numpy.zeros([N, N], dtype=complex)
vis_sub_o = vis[ilow:ihigh]
for i in range(len(vis_sub)):
    convgridone(grid_uv_o,(x[i], y[i]), (xf[i], yf[i]), wg, vis_sub[i])

ff = (1/T2/4) * numpy.mgrid[-ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j),
                            -ff_size:ff_size-2./over_sampling:(ff_size*over_sampling*1j)]
rot = numpy.exp(2j*numpy.pi*(ff[1] * delta_l + ff[0] * delta_m))


In [ ]:
(0.000977731015697-0.000803955164319j)